In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
import pinecone
from pinecone import Pinecone, ServerlessSpec

In [9]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [10]:
import json
data = json.load(open("reviews.json"))
data ['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Physics 101',
  'stars': 4,
  'review': 'Great professor! Explains concepts clearly and is always willing to help.'},
 {'professor': 'Dr. Emily Turner',
  'subject': 'Calculus II',
  'stars': 5,
  'review': 'Excellent lecturer, makes complex topics easy to understand.'},
 {'professor': 'Prof. Mark Johnson',
  'subject': 'Introduction to Philosophy',
  'stars': 3,
  'review': 'Interesting lectures but the grading is tough.'},
 {'professor': 'Dr. Laura Williams',
  'subject': 'Biology 201',
  'stars': 2,
  'review': "Content is challenging, but the professor isn't very engaging."},
 {'professor': 'Prof. David Lee',
  'subject': 'World History',
  'stars': 5,
  'review': 'Passionate about the subject and makes classes enjoyable.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Organic Chemistry',
  'stars': 4,
  'review': 'Difficult course but Dr. Thompson is very supportive.'},
 {'professor': 'Prof. James Brown',
  'subject': 'Computer 

In [13]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })


In [14]:
processed_data[0]


{'values': [-0.06027814,
  -0.037662193,
  -0.027879804,
  0.04329871,
  -0.038850054,
  -0.029719824,
  -0.021078715,
  0.036055084,
  -0.04579089,
  -0.023768872,
  0.008250978,
  -0.014626998,
  -0.010160873,
  -0.03013907,
  0.0110808825,
  0.03030211,
  0.0027396507,
  0.028671712,
  0.020869093,
  0.05296464,
  -0.0022083158,
  -0.005910192,
  0.018272102,
  0.04283288,
  -0.039246008,
  -0.02932387,
  -0.0113662025,
  0.025527373,
  0.01058594,
  0.0008537753,
  0.09069671,
  0.0034383927,
  -0.016513601,
  -0.042553384,
  -0.051753487,
  0.020962259,
  -0.009293268,
  -0.0015183081,
  -0.0009272888,
  0.01749184,
  -0.016094357,
  0.03214213,
  -0.019576421,
  0.015011306,
  0.05044917,
  0.0046000513,
  -0.059439648,
  -0.009700867,
  0.01616423,
  0.037010033,
  -0.0128684975,
  -0.0047805593,
  0.038570557,
  0.04348504,
  -0.03950221,
  0.029137539,
  0.036194835,
  -0.0027673093,
  0.006119815,
  -0.010294798,
  0.0297897,
  0.018458433,
  -0.0065972884,
  -0.010813031,
  

In [15]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

{'upserted_count': 20}

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}